In [6]:
import plot_function
from importlib import reload
reload(plot_function)
from plot_function import *

import matplotlib
matplotlib.pyplot.ioff()
%config InlineBackend.figure_formats = ['svg']
sns.set_theme(style="whitegrid")
# sns.set_theme(style="white")
sns.set_palette("colorblind")

In [7]:
job_details_template = {
    "scrap_measurement_overhead": {
        "id": 694
    },
    "moon": {
        "id": 804, # 3 Clients crashed,
        "dev_order": ["jao1", "jao2", "jon1", "jon2", "jon3", "jon4", "jxn1", "jxn2", "jn5", "jn6"],
        "dev_type_order": ["AGXOrin", "OrinNano", "XavierNX", "Nano"],
        "data_batches": [82, 57, 69, 78, 81, 59, 81, 67, 126, 76],
        "epochs": 10,
    },
    "moon_fedprox": {
        "id": 808,
        "max_round": 30,
        "inherits": "moon",
    },
    "moon_large_model": { 
        "id": 831, # confirm
        "dev_order": ["jao1", "jao2", "jon1", "jon2", "jon3", "jon4", "jxn1", "jxn2"],
        "dev_type_order": ["AGXOrin", "OrinNano", "XavierNX"],
        "data_batches": [82, 57, 69, 78, 81, 59, 81, 67, 126, 76],
        "max_round": 30, # max 31
        "epochs": 10,
    },
    "moon_large_model_opt": {
        "id": 832, 
        "max_round": 30, # max 45
        "inherits": "moon_large_model",
    },
    "moon_fedprox_large_model_opt": {
        "id": 833, 
        "max_round": 30, # max 48
        "inherits": "moon_large_model",
    },
    "test": {
        "id": 809
    },
}

def expand_inherits(job_details_template):
    expanded_data = {}
    
    for exp_name, job_details in job_details_template.items():
        if 'inherits' in job_details:
            inherited_job = job_details['inherits']
            expanded_details = {**job_details_template[inherited_job], **job_details}
            expanded_details.pop('inherits', None)
            expanded_data[exp_name] = expanded_details
        else:
            expanded_data[exp_name] = job_details
        
        # Add exp_name as field for exp
        expanded_data[exp_name]['exp_name'] = exp_name
    
    return expanded_data

job_details_map = expand_inherits(job_details_template)

In [ ]:
exp_name="scrap_measurement_overhead"
job_details = job_details_map[exp_name]
job_metric_dir = collect_job_metrics(job_details)
job_data = read_job_metrics(job_details, job_metric_dir)

# Adjust units
hw_metrics = job_data["hw_metrics"]
hw_metrics["Sent (KiB)"] = hw_metrics["Sent (MiB)"] * 1024
hw_metrics["Rcvd (KiB)"] = hw_metrics["Rcvd (MiB)"] * 1024
hw_metrics["Upload (KiB/s)"] = hw_metrics["Upload (MiB/s)"] * 1024
hw_metrics["Download (KiB/s)"] =  hw_metrics["Download (MiB/s)"]  * 1024 

# df = job_data["hw_metrics"][["device_type", "CPU Util (%)", "Mem Util (MiB)", "Power (W)"]]
df = job_data["hw_metrics"][["device_type", "CPU Util (%)", "Mem Util (MiB)"]]
plot_hw_metrics(df, save_name="scrape_hw_1.pdf")

# df = job_data["hw_metrics"][["device_type", "Sent (MiB)", "Rcvd (MiB)", "Upload (MiB/s)", "Download (MiB/s)"]]
df = job_data["hw_metrics"][["device_type", "Sent (KiB)", "Rcvd (KiB)", "Upload (KiB/s)", "Download (KiB/s)"]]
plot_hw_metrics(df, save_name="scrape_hw_2.pdf")


In [ ]:
exp_name="test"
job_details = job_details_map[exp_name]
job_metric_dir = collect_job_metrics(job_details)
order = ["JetsonAGXOrin", "JetsonOrinNano"]
job_data = read_job_metrics(job_details, job_metric_dir)
plot_cir_metrics(job_data["cr_timings"], job_details, order=order)

In [41]:
job_details = job_details_map["moon"]
moon_job_data = collect_job_metrics(job_details)
full_algo_plot(moon_job_data["cr_timings"], job_details)

/home/faustiar/colext/plotting/plot_function.py:141: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hw_metrics["time"] = pd.to_datetime(hw_metrics["time"], format='ISO8601')


In [15]:
job_details = job_details_map["moon_fedprox"]
moon_fedprox_job_data = collect_job_metrics(job_details)
full_algo_plot(moon_fedprox_job_data["cr_timings"], job_details)

In [8]:
job_details = job_details_map["moon_large_model"] 
moon_job_data = collect_job_metrics(job_details)
full_algo_plot(moon_job_data["cr_timings"], job_details, show=False)

Clipping rounds to 30


In [9]:
job_details = job_details_map["moon_large_model_opt"]
moon_job_data = collect_job_metrics(job_details)
full_algo_plot(moon_job_data["cr_timings"], job_details, show=False)

Clipping rounds to 30


In [10]:
job_details = job_details_map["moon_fedprox_large_model_opt"]
moon_job_data = collect_job_metrics(job_details)
full_algo_plot(moon_job_data["cr_timings"], job_details, show=False)

Clipping rounds to 30


In [13]:
algorithms = [
    ("moon", "Moon"),
    ("moon_fedprox", "FedProx"),
]

algorithms_to_cmp = []
for job_name, algorithm_title in algorithms:
    job_details = job_details_map[job_name]
    job_data = collect_job_metrics(job_details)["cr_timings"]
    job_data["Algorithm"] = algorithm_title
    algorithms_to_cmp.append(job_data)

cmp_algorithms_by_cir(algorithms_to_cmp, "device_type", save_file="cmp_moon_fedprox_orig.pdf")

/home/faustiar/colext/plotting/plot_function.py:147: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cr_timings: DataFrame = pd.read_csv(f"{path_prefix}_client_round_timings.csv", parse_dates=["start_time", "end_time"])


Clipping rounds to 30


In [21]:
algorithms = [
    ("moon_large_model", "Moon (L)"),
    ("moon_large_model_opt", "Moon (L+O)"),
    ("moon_fedprox_large_model_opt", "Fedprox (L)"),
]

algorithms_to_cmp = []
for job_name, algorithm_title in algorithms:
    job_details = job_details_map[job_name]
    job_data = collect_job_metrics(job_details)["cr_timings"]
    job_data["Algorithm"] = algorithm_title
    algorithms_to_cmp.append(job_data)

cmp_algorithms_by_cir(algorithms_to_cmp, "device_type", save_file="cmp_moon_fedprox_large.pdf")

Clipping rounds to 30
Clipping rounds to 30
Clipping rounds to 30


In [22]:
cmp_algorithms_by_cir(algorithms_to_cmp, "device_type", save_file="cmp_moon_fedprox_large.pdf")

In [ ]:

job_data["hw_metrics"].groupby(['client_id', 'round_number', 'stage']).describe()

In [41]:
job_data["hw_metrics"].groupby("device_name").mean(numeric_only=True)

,client_id,CPU Util (%),Mem Util (MiB),GPU Util (%),Power (W),Sent (MiB),Rcvd (MiB),Upload (MiB/s),Download (MiB/s),delta_t_sec,Energy (KJ)
device_name,,,,,,,,,,,
jao1,1.0,5.938010,1021.354482,14.989031,8.539661,0.217983,0.725861,0.008081,0.022612,0.299583,0.473790
jao2,0.0,6.054476,1003.134111,14.966496,8.254389,0.219824,0.718778,0.007040,0.022740,0.299619,0.454875
jon1,3.0,8.382969,1016.610995,29.496943,6.299031,0.291822,0.973631,0.011424,0.038031,0.299818,0.217313
jon4,2.0,9.090909,1005.616039,20.019394,5.657361,0.225645,0.852709,0.008250,0.026895,0.299414,0.267581
